<a href="https://colab.research.google.com/github/Byeon-MJ/ML_Practice_Repo/blob/main/Tree_Ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 트리 기반 앙상블 모델

## 랜덤 포레스트
* 결정의 앙상블이기 때문에 DecisionTree가 제공하는 중요한 매개변수를 제공
* 특성 중요도 계산 가능

### 데이터셋 준비

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()
train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42
)

In [2]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs=-1, random_state=42)

In [3]:
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


In [4]:
# Feature Importance 계산
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


In [5]:
# OOB Score 계산
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


## 엑스트라 트리
* 랜덤 포레스트와 매우 비슷하게 동작
* 결정 트리가 제공하는 대부분의 매개변수를 지원
* 랜덤 포레스트와의 차이점은 부트스트랩 샘플을 사용하지 않는다는 점
* 각 결정 트리를 만들 때 전체 훈련 세트를 사용함
* 노드를 랜덤하게 분할하기 때문에 계산속도가 빠르다


In [12]:
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target,
                        return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


In [13]:
# 특성 중요도 계산
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


## 그래디언트 부스팅
* 과적합에 강한 모습

In [15]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target,
                        return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


In [16]:
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2,
                                random_state=42)
scores = cross_validate(gb, train_input, train_target,
                        return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [17]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15872278 0.68010884 0.16116839]


## 히스토그램 기반 그래디언트 부스팅
* 입력 특성을 256개의 구간으로 나눠서 노드 분할시 최적의 분할을 빠르게 찾을 수 있다.
* 256개의 구간 중에서 하나를 떼어 놓고 누락된 값을 위해서 사용
* 입력이 누락된 특성이 있더라도 따로 전처리할 필요가 없음

In [20]:
from sklearn.ensemble import HistGradientBoostingClassifier
hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


### Feature Importance 계산 : permutation_importance()
* 특성을 하나씩 랜덤하게 섞어서 모델의 성능이 변화하는지를 관찰
* 훈련 세트 뿐만 아니라 테스트 세트에도 적용 가능
* permutation_importance() 함수가 반환하는 객체는 중요도(importances), 평균(importances_mean), 표준 편차(importances_std) 를 담고있음

In [22]:
# 훈련셋 Feature Importance 계산
from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target,
                                n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


In [23]:
# 테스트셋 Feature Importance 계산
result = permutation_importance(hgb, test_input, test_target,
                                n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


In [24]:
hgb.score(test_input, test_target)

0.8723076923076923

## XGBoost

In [26]:
from xgboost import XGBClassifier
xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target, return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8824322471423747 0.8726214185237284


## LightGBM

In [28]:
from lightgbm import LGBMClassifier
lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9338079582727165 0.8789710890649293
